In [3]:
from pygame.locals import *

import pygame
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *


def cube():
    vertices = (
        (1,-1,-1),
        (1,1,-1),
        (-1,1,-1),
        (-1,-1,-1),
        (1,-1,1),
        (1,1,1),
        (-1,-1,1),
        (-1,1,1)
    )
    edegs = (
        (0,1),
        (0,3),
        (0,4),
        (2,1),
        (2,3),
        (2,7),
        (6,3),
        (6,4),
        (6,7),
        (5,1),
        (5,4),
        (5,7),
    )
    glBegin(GL_LINES)
    for edeg in edegs:
        for vertex in edeg:
            glVertex3fv(vertices[vertex])
    glEnd()

def text(x, y, text):                                                
    textSurface = pygame.font.SysFont('arial', 20).render(str(text), True, (255, 255, 66, 255), (0, 66, 0, 255))
    textData = pygame.image.tostring(textSurface, "RGBA", True)
    glWindowPos2d(x, y)
    glDrawPixels(textSurface.get_width(), textSurface.get_height(), GL_RGBA, GL_UNSIGNED_BYTE, textData)

pygame.init()
screen = (800,600)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)
glMatrixMode(GL_PROJECTION)

gluPerspective(45, (screen[0]/screen[1]), 0.1, 50.0)

glTranslate(0.0,0.0,-5)
glMatrixMode(GL_MODELVIEW)  
modelMatrix = glGetFloatv(GL_MODELVIEW_MATRIX)

busy = True
button_down = False
while busy:
    glPushMatrix()
    glLoadIdentity()

    for event in pygame.event.get():
        if(event.type == pygame.QUIT):
            pygame.quit()
            busy = False
        elif event.type == pygame.MOUSEMOTION:
            if(button_down):
                glRotatef(event.rel[1], 1, 0, 0)
                glRotatef(event.rel[0], 0, 1, 0)
    
    for event in pygame.mouse.get_pressed():
        if pygame.mouse.get_pressed()[0] == 1:
            button_down = True
        elif pygame.mouse.get_pressed()[0] == 0:
            button_down = False
            
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    glMultMatrixf(modelMatrix)
    modelMatrix = glGetFloatv(GL_MODELVIEW_MATRIX)

    glLoadIdentity()
    glTranslatef(0, 0, -5)
    glMultMatrixf(modelMatrix)

    cube()

    
    glPopMatrix()
    
        

    pygame.display.flip()
    pygame.time.wait(10)
    
    

error: video system not initialized